# ConversationEntityMemory

- Author: [ulysyszh](https://github.com/ulysyszh)
- Design: [ulysyszh](https://github.com/ulysyszh)
- Peer Review: [Kenny Jung](https://www.linkedin.com/in/kwang-yong-jung)
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ulysyszh/LangChain-OpenTutorial/blob/main/05-Memory/04_ConversationEntityMemory.ipynb)


## Overview

`ConversationEntityMemory` allows the conversation system to retain facts about specific entities mentioned during the dialogue.

In this case, Extracting information about entities from the conversation using a large language model.

Accumulating knowledge about these entities over time using the same large language model.


### Table of Contents

- [Overview](#overview)
- [Environment Setup](#environment-setup)
- [Conversation-Based Entity Memory Example](#Conversation-based-entity-memory-example)
- [Retrieving Entity Memory](#retrieving-entity-memory)

### References

- [LangChain Python API Reference > langchain: 0.3.13 > memory > ConversationEntityMemory](https://python.langchain.com/api_reference/langchain/memory/langchain.memory.entity.ConversationEntityMemory.html)
----

## Environment Setup
Set up the environment to use ConversationEntityMemory.

Installing Required Libraries
Install the necessary dependencies using the langchain-opentutorial package:

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials.
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.


In [38]:
!pip install langchain langchain-community langchain-opentutorial

In [39]:
# Install required packages
from langchain_community.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory.entity import ConversationEntityMemory
from langchain_opentutorial import package

openai = ChatOpenAI(model="gpt-4o-mini")

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_core",
        "langchain-anthropic",
        "langchain_community",
        "langchain_text_splitters",
        "langchain_openai",
    ],
    verbose=False,
    upgrade=False,
)

In [45]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "ConversationEntityMemory",
    }
)

Environment variables have been set successfully.


In [46]:
from dotenv import load_dotenv

load_dotenv(override=True)

False

You can alternatively set `OPENAI_API_KEY` in `.env` file and load it.

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [51]:
from langchain.prompts import PromptTemplate

entity_memory_conversation_template = PromptTemplate(
    input_variables=["entities", "history", "input"],
    template="""
You are an assistant to a human, powered by a large language model trained by OpenAI.

You assist with various tasks, from answering simple questions to providing detailed discussions on a wide range of topics. You can generate human-like text, allowing natural conversations and coherent, relevant responses.

You constantly learn and improve, processing large amounts of text to provide accurate and informative responses. You can use personalized information provided in the context below, along with your own generated knowledge.

Context:
{entities}

Current conversation:
{history}
Last line:
Human: {input}
You:
"""
)

print(entity_memory_conversation_template)

input_variables=['entities', 'history', 'input'] input_types={} partial_variables={} template='\nYou are an assistant to a human, powered by a large language model trained by OpenAI.\n\nYou assist with various tasks, from answering simple questions to providing detailed discussions on a wide range of topics. You can generate human-like text, allowing natural conversations and coherent, relevant responses.\n\nYou constantly learn and improve, processing large amounts of text to provide accurate and informative responses. You can use personalized information provided in the context below, along with your own generated knowledge.\n\nContext:\n{entities}\n\nCurrent conversation:\n{history}\nLast line:\nHuman: {input}\nYou:\n'


## Conversation-Based Entity Memory Example

This example demonstrates how to use 'ConversationEntityMemory' to store and manage information about entities mentioned during a conversation. The conversation accumulates ongoing knowledge about these entities while maintaining a natural flow.

In [52]:
llm = ChatOpenAI(temperature=0)

conversation = ConversationChain(
    llm=llm,
    prompt=entity_memory_conversation_template,
    memory=ConversationEntityMemory(llm=llm),
)

## Retrieving Entity Memory

Let's examine the conversation history stored in memory using the `load_memory_variables()` method to verify our window-based memory retention.

In [53]:
# Input conversation
response = conversation.predict(
    input=(
        "Amelia is an award-winning landscape photographer who has traveled around the globe capturing natural wonders. "
        "David is a wildlife conservationist dedicated to protecting endangered species. "
        "They are planning to open a nature-inspired photography gallery and learning center that raises funds for conservation projects."
    )
)

# Print the assistant's response
print(response)

That sounds like a wonderful initiative! Amelia's expertise in landscape photography combined with David's passion for wildlife conservation will surely create a unique and impactful gallery and learning center. By showcasing the beauty of nature through photography, they can raise awareness and support for conservation efforts. It's inspiring to see individuals using their talents to make a positive difference in the world.


In [54]:
# Print the entity memory
conversation.memory.entity_store.store

{'Amelia': 'Amelia is an award-winning landscape photographer who has traveled around the globe capturing natural wonders.',
 'David': 'David is a wildlife conservationist dedicated to protecting endangered species.'}